In [1]:
# Librerias
import os
import re
import time
import glob
import json
import random
import numpy as np
import pandas as pd
from pathlib import Path
from datetime import datetime
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions as EC

In [2]:
archivo_rucs = "ListaRucs.txt"
archivo_imposibles = 'RucsProcesados.txt'

def cargar_rucs(path):
    if os.path.exists(path):
        with open(path, 'r', encoding='utf-8') as f:
            return set(line.strip() for line in f if line.strip())
    return set()

Rucs = cargar_rucs(archivo_rucs)
Rucs = pd.DataFrame(list(Rucs), columns=['Rucs'])
RucProcesados = cargar_rucs(archivo_imposibles)

Rucs = Rucs[~Rucs['Rucs'].astype(str).isin(set(RucProcesados))]
Rucs = Rucs[Rucs['Rucs'].astype(str).str.match(r'^\d{11}$')].reset_index(drop=True)

print(f"Faltan procesar {len(Rucs)} Rucs")

Faltan procesar 0 Rucs


In [3]:
rutaOesce = os.path.join(os.getcwd(), 'RegistroOece')
for index, row in Rucs.iterrows():
    ruc = str(row['Rucs'])
    print(f"Obteniendo HTML para el RUC: {ruc}")

    URL = "https://eap.oece.gob.pe/perfilprov-bus/1.0/ficha/" + ruc + "/contrataciones/exportar"

    time.sleep(10)

    driver = webdriver.Chrome()
    driver.get(URL)
    
    time.sleep(5)
    
    html_completo = driver.page_source
    nombre_archivo = f'procesado_{ruc}.html'
    ruta_archivo = os.path.join(rutaOesce, nombre_archivo)

    with open(rutaOesce, 'w', encoding='utf-8') as f:
        f.write(html_completo)

    with open(archivo_imposibles, 'a', encoding='utf-8') as f:
        f.write(f"{ruc}\n")

    try:
        driver.quit()
    except Exception as e:
        print(f"Error al cerrar el navegador para el RUC {ruc}: {e}")

In [4]:
rutaOesce = os.path.join(os.getcwd(), 'RegistroOece')
os.makedirs(rutaOesce, exist_ok=True)

lista_dfs = []

for archivo in os.listdir(rutaOesce):
    if archivo.endswith('.html'):
        ruta_archivo = os.path.join(rutaOesce, archivo)
        with open(ruta_archivo, 'r', encoding='utf-8') as f:
            soup = BeautifulSoup(f, 'html.parser')
            data_json = json.loads(soup.find('pre').text)
            if 'contratosE01' in data_json:
                df = pd.DataFrame(data_json['contratosE01'])
                ruc = archivo.replace('procesado_', '').replace('.html', '')
                df['RUC Origen'] = ruc
                lista_dfs.append(df)

df_consolidado = pd.concat(lista_dfs, ignore_index=True)

def obtener_porcentaje(ruc_objetivo, miembros):
    if pd.isna(miembros):
        return None
    pares = miembros.split('||')
    for par in pares:
        partes = par.split('|')
        if len(partes) >= 2 and partes[0] == ruc_objetivo:
            try:
                return float(partes[1])
            except ValueError:
                return None
    return None

df_consolidado['monto_del_contrato_original'] = pd.to_numeric(
    df_consolidado['monto_del_contrato_original'], errors='coerce')

df_consolidado['participacion_ruc_origen'] = df_consolidado.apply(
    lambda row: obtener_porcentaje(row['RUC Origen'], row.get('miembros_consorcio')),
    axis=1)

df_consolidado['Valor Proporcional GE'] = df_consolidado['participacion_ruc_origen'] * df_consolidado['monto_del_contrato_original'] /100

df_consolidado['fecha_de_firma_de_contrato'] = pd.to_datetime(df_consolidado['fecha_de_firma_de_contrato'], dayfirst=True, errors='coerce')
df_consolidado['fecha_prevista_de_fin_de_contrato'] = pd.to_datetime(df_consolidado['fecha_prevista_de_fin_de_contrato'], dayfirst=True, errors='coerce')

df_consolidado['Nro de dias'] = (
    df_consolidado['fecha_prevista_de_fin_de_contrato'] - df_consolidado['fecha_de_firma_de_contrato']).dt.days

df_consolidado['Valor Mensual proporcional'] = (df_consolidado['Valor Proporcional GE']/df_consolidado['Nro de dias'])*30

df_consolidado['plazo en Meses'] = round(df_consolidado['Nro de dias'] /30,1)

df_consolidado['Valor Proporcional GE'] = df_consolidado['Valor Proporcional GE'].apply(
    lambda x: f"{x:,.2f}" if pd.notna(x) else "")

df_consolidado['Valor Mensual proporcional'] = df_consolidado['Valor Mensual proporcional'].apply(
    lambda x: f"{x:,.2f}" if pd.notna(x) else "")

df_consolidado.to_parquet("Consolidado.parquet", index=False)